# 4.1 Feature Engineering & 최종 데이터 생성

- 각 카테고리별, 결측치 처리 방법별 모델링에 사용할 데이터 생성
- 이 데이터를 사용하여 `5_LG_고객데이터_결측치_방법론선택.ipynb` 파일에서 각 카테고리에 맞는 결측치 처리 방법 선택

In [ ]:
import pandas as pd
from tqdm import tqdm
from datetime import datetime

pd.set_option('display.max_columns', None)


In [148]:
### 결측치 완료한 데이터 프레임으로 수정 필요
# df = pd.read_csv('../data/결측치_output/결측치_f1기준_v1.csv', index_col = 0)

In [202]:
# df = pd.read_csv('../data/결측치_output/결측치_CategoricalImputer.csv', index_col = 0)

In [ ]:
# df = pd.read_csv('../data/결측치_output/결측치_RandomSampleImputer.csv', index_col = 0)

In [ ]:
# df = pd.read_csv('../data/결측치_ArbitaryImputer.csv', index_col = 0)

In [458]:
# df = pd.read_csv('../data/결측치_output/결측치_ArbitaryImputer.csv', index_col = 0)
# df = pd.read_csv('../data/결측치_output/결측치_KNNImputer_distance10_final.csv', index_col = 0)

## 2023년에 구매한 이력은 제거

In [203]:
df.loc[df['년도'] == 2023, 'label_2023'] = 1
df.loc[df['년도'] != 2023, 'label_2023'] = 0

In [204]:
df_isbuy_2023 = df[['고객ID', 'label_2023']]
df_isbuy_2023 = df_isbuy_2023[df_isbuy_2023['label_2023'] == 1]
df_isbuy_2023.drop_duplicates("고객ID", inplace = True)
df_isbuy_2023.rename({'label_2023':'label_y'}, axis = 1, inplace = True)
df_isbuy_2023

,고객ID,label_y
6250,40445,1.0
9260,24253,1.0
12405,63417,1.0
16373,22718,1.0
18475,20032,1.0
...,...,...
532602,91470,1.0
532609,92672,1.0
532612,98487,1.0
532616,102635,1.0


In [206]:
df['label_2023'].value_counts()

0.0    503612
1.0     29013
Name: label_2023, dtype: int64

In [207]:
# 꼭 돌려야함
df = df.loc[df['label_2023'] != 1]

## 제품군 별로 나누기

In [210]:
def split_category(x):
    if x in ['냉장고','김치냉장고', '정수기', '식기세척기', '전기레인지', '얼음정수기냉장고', '광파오븐']:
        new_c = '부엌'
    elif x in ['TV', 'PC', '모니터', 'AV', '휴대폰']:
        new_c = '디스플레이'
    elif x in ['세탁기', '청소기', '건조기', '스타일러', '미니워시', '워시타워', '로봇청소기']:
        new_c = '청소'
    elif x in ['뷰티기기', '안마의자']:
        new_c = '건강관리'
    elif x in ['에어컨', '공기청정기']:
        new_c = '에어솔루션'
    else:
        new_c = '기타'
    return new_c

In [211]:
df['제품카테고리'] = df['타겟제품명'].apply(lambda x: split_category(x))

In [212]:
df[df['제품카테고리'] == '기타']['타겟제품명'].unique()

array(['시그니처TV', '시그니처냉장고', '시그니처세탁기', '시그니처공청기', '시그니처기타', '시그니처에어컨'],
      dtype=object)

## 가공 변수 추가

- df_crm에 가공변수를 추가 
- 마지막에 df_crm과 제품별로 나눠진 데이터를 merge 필요

In [213]:
df_crm = pd.read_csv('../data/master_crm.csv', encoding='cp949')

### 총구매빈도

In [214]:
# df.loc[df['년도'] == 2023, 'label_2023'] = 1
# df.loc[df['년도'] != 2023, 'label_2023'] = 0

In [215]:
df_crm

,고객ID,멤버십고객여부,케어솔루션고객여부,케어솔루션품목수,케어십가입여부,케어십품목수,최근3년구매금액,최근구매채널,Only&Best품목수,건강관리가전품목수,연령대,주거형태,아파트시세,아파트평형대,포인트사용빈도,보유멤버십포인트,멤버십가입경로,제휴카드보유여부,프리미엄카드보유여부,제휴가입여부,멤버십앱설치여부,거주광역명,거주시군구명,거주행정동명,label
0,1,NaN,0.0,0품목,1,2품목,5KK이상10KK미만,홈플러스,2품목,2품목,NaN,아파트,1억↓,20평대,미사용,NaN,NaN,0,0,0,NaN,경기,안산시 상록구,일동,0
1,2,NaN,0.0,1품목,0,0품목,1KK미만,B2B,2품목,2품목,NaN,아파트,3~4억,40평대,미사용,NaN,NaN,0,0,0,NaN,경기,오산시,대원동,0
2,3,NaN,1.0,0품목,0,0품목,0원,하이마트,1품목,2품목,NaN,아파트,5~6억,30평대,미사용,NaN,NaN,0,0,0,NaN,경기,화성시,새솔동,0
3,4,NaN,0.0,3품목,0,0품목,1KK미만,베스트샵,1품목,2품목,NaN,아파트,1~2억,30평대,미사용,NaN,NaN,0,0,0,NaN,전북,전주시 덕진구,여의동,0
4,5,NaN,0.0,0품목,1,2품목,1KK이상3KK미만,이마트,1품목,2품목,NaN,아파트,3~4억,40평대,미사용,NaN,NaN,0,0,0,NaN,경기,용인시 처인구,역북동,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104347,104348,0.0,0.0,1품목,0,0품목,1KK미만,온라인,1품목,1품목,30대,아파트,11~15억,20평대,미사용,NaN,LGE회원통합,0,0,0,0.0,서울,양천구,신정1동,1
104348,104349,0.0,NaN,0품목,0,0품목,1KK이상3KK미만,베스트샵,0품목,0품목,30대,아파트,1억↓,20평대,미사용,NaN,LGE회원통합,0,0,0,0.0,충북,충주시,칠금.금릉동,1
104349,104350,0.0,0.0,1품목,1,1품목,1KK이상3KK미만,베스트샵,1품목,2품목,30대,아파트,3~4억,30평대,미사용,NaN,모닝글로리,0,0,0,1.0,경기,수원시 권선구,평동,1
104350,104351,0.0,0.0,0품목,0,0품목,1KK이상3KK미만,베스트샵,1품목,1품목,40대,아파트,7~8억,30평대,미사용,NaN,스마트월렛,0,0,0,0.0,서울,도봉구,창1동,1


In [216]:
df_crm = df_crm.merge(df.groupby(['고객ID']).count()['label_2023'], on = "고객ID")
df_crm = df_crm.rename(columns = {"label_2023" : "총구매빈도"})
df_crm.head(3)

,고객ID,멤버십고객여부,케어솔루션고객여부,케어솔루션품목수,케어십가입여부,케어십품목수,최근3년구매금액,최근구매채널,Only&Best품목수,건강관리가전품목수,연령대,주거형태,아파트시세,아파트평형대,포인트사용빈도,보유멤버십포인트,멤버십가입경로,제휴카드보유여부,프리미엄카드보유여부,제휴가입여부,멤버십앱설치여부,거주광역명,거주시군구명,거주행정동명,label,총구매빈도
0,1,NaN,0.0,0품목,1,2품목,5KK이상10KK미만,홈플러스,2품목,2품목,NaN,아파트,1억↓,20평대,미사용,NaN,NaN,0,0,0,NaN,경기,안산시 상록구,일동,0,3
1,2,NaN,0.0,1품목,0,0품목,1KK미만,B2B,2품목,2품목,NaN,아파트,3~4억,40평대,미사용,NaN,NaN,0,0,0,NaN,경기,오산시,대원동,0,3
2,3,NaN,1.0,0품목,0,0품목,0원,하이마트,1품목,2품목,NaN,아파트,5~6억,30평대,미사용,NaN,NaN,0,0,0,NaN,경기,화성시,새솔동,0,5


### 최대구매가격대

In [218]:
df['금액_중간값'] = df['금액'].replace(to_replace={"Missing": 0, "0만원 이상 100만원 이하":50 , "100만원 이상 200만원 이하":150, "200만원 이상 300만원 이하":250, "300만원 이상 400만원 이하":350, "400만원 이상 500만원 이하":450, "500만원 이상 600만원 이하":550, "600만원 이상 700만원 이하":650, "700만원 이상 800만원 이하":750, "800만원 이상 900만원 이하":850, "900만원 이상 1000만원 이하":950, "1000만원 이상":1000 })

In [219]:
# 고객별로 그룹화하 후 각 그룹에서 최대값 리턴
max_purchase_by_customer = df.groupby('고객ID')['금액_중간값'].max()

In [220]:
df_crm['최대구매가격대'] = max_purchase_by_customer.reset_index(drop = True)
df_crm.head(2)

,고객ID,멤버십고객여부,케어솔루션고객여부,케어솔루션품목수,케어십가입여부,케어십품목수,최근3년구매금액,최근구매채널,Only&Best품목수,건강관리가전품목수,연령대,주거형태,아파트시세,아파트평형대,포인트사용빈도,보유멤버십포인트,멤버십가입경로,제휴카드보유여부,프리미엄카드보유여부,제휴가입여부,멤버십앱설치여부,거주광역명,거주시군구명,거주행정동명,label,총구매빈도,최대구매가격대
0,1,NaN,0.0,0품목,1,2품목,5KK이상10KK미만,홈플러스,2품목,2품목,NaN,아파트,1억↓,20평대,미사용,NaN,NaN,0,0,0,NaN,경기,안산시 상록구,일동,0,3,250
1,2,NaN,0.0,1품목,0,0품목,1KK미만,B2B,2품목,2품목,NaN,아파트,3~4억,40평대,미사용,NaN,NaN,0,0,0,NaN,경기,오산시,대원동,0,3,150


### 가장 최근 구매 이후 경과일

In [221]:
df['매출일자(배송완료일자)'] = pd.to_datetime(df['매출일자(배송완료일자)'])
current_date = datetime.now()

# 각 고객 별로 최근 매출일자(배송완료일자)를 찾고 일수 차이 계산
customer_last_purchase_dates = df.groupby('고객ID')['매출일자(배송완료일자)'].max().reset_index()
customer_last_purchase_dates['최근매출일로부터경과일'] = (current_date - customer_last_purchase_dates['매출일자(배송완료일자)']).dt.days

customer_last_purchase_dates

,고객ID,매출일자(배송완료일자),최근매출일로부터경과일
0,1,2022-01-11,707
1,2,2021-12-29,720
2,3,2020-08-17,1219
3,4,2021-06-05,927
4,5,2020-10-08,1167
...,...,...,...
102686,104347,2022-12-21,363
102687,104348,2020-12-29,1085
102688,104349,2022-12-24,360
102689,104350,2018-06-11,2017


In [222]:
df_crm = df_crm.merge(customer_last_purchase_dates[['고객ID', '매출일자(배송완료일자)', '최근매출일로부터경과일']], on = "고객ID")

### 가장 선호하는 구매 제품

In [223]:
# 각 고객ID 별로 제품 카테고리의 구매 횟수를 계산
customer_purchase_counts = df.groupby(['고객ID', '제품카테고리']).size().reset_index(name='구매횟수')

# 각 고객ID 별로 가장 높은 구매 횟수를 찾아 해당하는 제품 카테고리를 반환
max_purchase_categories = customer_purchase_counts.groupby('고객ID').apply(lambda x: x[x['구매횟수'] == x['구매횟수'].max()]['제품카테고리'].tolist())

max_purchase_categories

고객ID
1                    [부엌]
2                    [부엌]
3                    [부엌]
4         [부엌, 에어솔루션, 청소]
5                    [청소]
               ...       
104347            [디스플레이]
104348        [부엌, 에어솔루션]
104349        [디스플레이, 부엌]
104350            [에어솔루션]
104351        [디스플레이, 청소]
Length: 102691, dtype: object

In [224]:
df_crm['주구매제품카테고리'] = max_purchase_categories.reset_index(drop = True)
df_crm.head(2)

,고객ID,멤버십고객여부,케어솔루션고객여부,케어솔루션품목수,케어십가입여부,케어십품목수,최근3년구매금액,최근구매채널,Only&Best품목수,건강관리가전품목수,연령대,주거형태,아파트시세,아파트평형대,포인트사용빈도,보유멤버십포인트,멤버십가입경로,제휴카드보유여부,프리미엄카드보유여부,제휴가입여부,멤버십앱설치여부,거주광역명,거주시군구명,거주행정동명,label,총구매빈도,최대구매가격대,매출일자(배송완료일자),최근매출일로부터경과일,주구매제품카테고리
0,1,NaN,0.0,0품목,1,2품목,5KK이상10KK미만,홈플러스,2품목,2품목,NaN,아파트,1억↓,20평대,미사용,NaN,NaN,0,0,0,NaN,경기,안산시 상록구,일동,0,3,250,2022-01-11,707,[부엌]
1,2,NaN,0.0,1품목,0,0품목,1KK미만,B2B,2품목,2품목,NaN,아파트,3~4억,40평대,미사용,NaN,NaN,0,0,0,NaN,경기,오산시,대원동,0,3,150,2021-12-29,720,[부엌]


# 방법론 1 (선택)
- Idea : 각 제품군 별로 특징이 다를 것이므로, 제품군 별로 구매가능성을 예측하는 데에 더 정확할 것
- 따라서, 제품 카테고리 별로 데이터프레임을 구분한 후, 각각 제품군 별로 모델링을 하여, 모델을 여러 개 생성
- 이때, Y_target은 각 제품군 구매여부(1, 0)가 됨.
    - 2023년의 구매이력과 매칭하여, 어떤 제품군 카테고리를 구매했는지 확인할 수 있으므로, 각 제품군 별 구매여부 추적이 가능함

기본컬럼 (23) : '고객ID', '멤버십고객여부', '케어솔루션품목수', '케어십가입여부',
       '케어십품목수', '최근3년구매금액', '최근구매채널', 'Only&Best품목수', '건강관리가전품목수', '연령대',
       '주거형태', '아파트시세', '아파트평형대', '포인트사용빈도', '보유멤버십포인트', '멤버십가입경로', '제휴카드보유여부',
       '프리미엄카드보유여부', '제휴가입여부', '멤버십앱설치여부', '거주광역명', '거주시군구명', 'label_y'
       
추가 (4의배수) : '년도', '월', '타겟제품명', '금액'

가공변수 (4) : 총구매빈도, 최대구매가격대, 주구매제품카테고리

모델링에서 제거 : '최근매출일로부터경과일', '고객ID', '케어솔루션고객여부', '거주행정동명'

## 제품별로 데이터프레임 생성

### 모든 데이터

In [225]:
df_isbuy_2023

,고객ID,label_y
6250,40445,1.0
9260,24253,1.0
12405,63417,1.0
16373,22718,1.0
18475,20032,1.0
...,...,...
532602,91470,1.0
532609,92672,1.0
532612,98487,1.0
532616,102635,1.0


In [226]:
df_y = df.merge(df_isbuy_2023, on = '고객ID', how = 'left')
df_y.loc[df_y['label_y'].isnull(), 'label_y'] = 0

In [227]:
df_y['label_y'].value_counts()

0.0    443383
1.0     60229
Name: label_y, dtype: int64

In [228]:
df_y['년도'].unique()

array([2022, 2020, 2016, 2015, 2019, 2017, 2021, 2010, 2014, 2018, 2012,
       2011, 2013, 2009])

### Air Solution

In [229]:
air_df = df_y.loc[df_y['제품카테고리'] == '에어솔루션']

In [230]:
# air_df.loc[:, 'label_y'] = 0
# air_df.loc[(air_df['년도'] == 2023) & (air_df['제품카테고리'] == '에어솔루션'), 'label_y'] = 1

In [231]:
air_df['label_y'].value_counts()

0.0    148739
1.0     19017
Name: label_y, dtype: int64

In [232]:
# air_df

### Clean

In [233]:
clean_df = df_y.loc[df_y['제품카테고리'] == '청소']

In [234]:
clean_df['label_y'].value_counts()

0.0    109934
1.0     15473
Name: label_y, dtype: int64

### Kitchen

In [235]:
kitchen_df = df_y.loc[df_y['제품카테고리'] == '부엌']


In [236]:
kitchen_df['label_y'].value_counts()

0.0    90839
1.0    12960
Name: label_y, dtype: int64

### Display

In [237]:
display_df = df_y.loc[df_y['제품카테고리'] == '디스플레이']
# display_df.loc[:, 'label_y'] = 0
# display_df.loc[(display_df['년도'] == 2023) & (display_df['제품카테고리'] == '디스플레이'), 'label_y'] = 1

In [238]:
display_df['label_y'].value_counts()

0.0    86956
1.0    11774
Name: label_y, dtype: int64

### Health

In [239]:
health_df = df_y.loc[df_y['제품카테고리'] == '건강관리']

In [240]:
health_df['label_y'].value_counts()

0.0    6486
1.0     922
Name: label_y, dtype: int64

### Signature

In [241]:
etc_df = df_y.loc[df_y['제품카테고리'] == '기타']

In [242]:
etc_df['label_y'].value_counts()

0.0    429
1.0     83
Name: label_y, dtype: int64

## 고객의 구매이력 데이터 통합 함수

In [243]:
def create_row_from_group(group, customer_id):

    new_row = {'고객ID': customer_id}
    for column in group.columns:
        new_row[column] = group.iloc[0][column]
    return new_row

In [244]:
def organize_purchases_per_customer(dataframe, column_list):
    
    # 단일 고객의 최대 구매 횟수 파악
    grouped_new = dataframe.groupby('고객ID')
    max_purchases_new = grouped_new.size().max()

    label_encoding_df = {column: [] for column in column_list}

    for i in range(1, max_purchases_new + 1):
        label_encoding_df[f'년도_{i}'] = []
        label_encoding_df[f'월_{i}'] = []
        label_encoding_df[f'타겟제품명_{i}'] = []
        label_encoding_df[f'금액_{i}'] = []

    for customer_id, group in tqdm(grouped_new):
        purchases = list(group[['년도', '월','타겟제품명', '금액']].itertuples(index=False, name=None))
        new_row_new = create_row_from_group(group, customer_id)

        # 새 행에 판매 날짜 및 구매 추가
        for i, (year, month, product_name, amount) in enumerate(purchases, 1):
            new_row_new[f'년도_{i}'] = year
            new_row_new[f'월_{i}'] = month
            new_row_new[f'타겟제품명_{i}'] = product_name
            new_row_new[f'금액_{i}'] = amount

        # 최대 구매수보다 적은 경우 나머지 행 채우기
        for j in range(len(purchases) + 1, max_purchases_new + 1):
            new_row_new[f'년도_{j}'] = None
            new_row_new[f'월_{j}'] = None
            new_row_new[f'타겟제품명_{j}'] = None
            new_row_new[f'금액_{j}'] = None

        for key in label_encoding_df:
            label_encoding_df[key].append(new_row_new.get(key, None))

    new_structured_df_new = pd.DataFrame(label_encoding_df)
    return new_structured_df_new


## 고객별 데이터 생성 : 고객 정보와 가공변수 포함된 df 

In [245]:
def final_df_maker(dataframe, col_list):
    df_new = organize_purchases_per_customer(dataframe, col_list)
    # 가공변수 추가
    df_new = df_new.merge(df_crm[['고객ID','총구매빈도', '최대구매가격대',
       '주구매제품카테고리']], on = '고객ID', how = 'left')
    return df_new

In [246]:
col_list = [
'고객ID','멤버십고객여부', '케어솔루션품목수', '케어십가입여부', '케어십품목수', '최근3년구매금액', '최근구매채널',
'Only&Best품목수', '건강관리가전품목수', '연령대', '주거형태', '아파트시세', '아파트평형대', '포인트사용빈도',
'보유멤버십포인트', '멤버십가입경로', '제휴카드보유여부', '프리미엄카드보유여부', '제휴가입여부', '멤버십앱설치여부',
'거주광역명', '거주시군구명', 'label_y']

In [247]:
etc_df_new = final_df_maker(etc_df, col_list)

100%|████████████████████████████████████████| 397/397 [00:01<00:00, 384.05it/s]


In [248]:
health_df_new = final_df_maker(health_df, col_list)

100%|██████████████████████████████████████| 5039/5039 [00:13<00:00, 384.40it/s]


In [249]:
clean_df_new = final_df_maker(clean_df, col_list)

100%|████████████████████████████████████| 72873/72873 [03:06<00:00, 390.53it/s]


In [252]:
air_df_new = final_df_maker(air_df, col_list)

100%|████████████████████████████████████| 63547/63547 [02:42<00:00, 390.58it/s]


In [253]:
kitchen_df_new = final_df_maker(kitchen_df, col_list)
display_df_new = final_df_maker(display_df, col_list)

100%|████████████████████████████████████| 62058/62058 [02:41<00:00, 383.93it/s]


In [254]:
all_df_new = final_df_maker(df_y, col_list)

100%|██████████████████████████████████| 102691/102691 [04:21<00:00, 392.52it/s]


In [ ]:
# etc_df_new.to_csv('../data/etc/etc_df_최빈값_2023제거.csv', encoding='utf-8-sig')


In [255]:
# etc_df_new.to_csv('../data/etc/etc_df_최빈값_2023제거.csv', encoding='utf-8-sig')
# health_df_new.to_csv('../data/health/health_df_최빈값_2023제거.csv', encoding='utf-8-sig')
# air_df_new.to_csv('../data/air/air_df_최빈값_2023제거.csv', encoding='utf-8-sig')
# kitchen_df_new.to_csv('../data/kitchen/kitchen_df_최빈값_2023제거.csv', encoding='utf-8-sig')
# display_df_new.to_csv('../data/display/display_df_최빈값_2023제거.csv', encoding='utf-8-sig')
# clean_df_new.to_csv('../data/clean/clean_df_최빈값_2023제거.csv', encoding='utf-8-sig')


In [256]:
# all_df_new.to_csv('../data/all/all_df_최빈값_2023제거.csv', encoding='utf-8-sig')

## 미세먼지, 온도 추가

In [117]:
df_x = df_y.copy()

### 미세먼지

In [118]:
dust1 = pd.read_excel('../data/외부변수/20091120-20181231_미세먼지농도.xlsx')

In [119]:
dust2 = pd.read_excel('../data/외부변수/20190101-20230831_미세먼지농도.xlsx')

In [120]:
dust = pd.concat([dust1, dust2])

In [121]:
dust

,지점,지점명,일시,일 미세먼지 농도(㎍/㎥)
0,90,속초,2009-11-20,25
1,90,속초,2009-11-21,27
2,90,속초,2009-11-22,19
3,90,속초,2009-11-23,48
4,90,속초,2009-11-24,63
...,...,...,...,...
41918,501,연평도,2023-08-27,34
41919,501,연평도,2023-08-28,19
41920,501,연평도,2023-08-29,18
41921,501,연평도,2023-08-30,12


In [122]:
mean_dust = dust[['일시', '일 미세먼지 농도(㎍/㎥)']].groupby('일시').mean().reset_index()

### 위의 일 미세먼지 농도가 결측치인 것을 '서울시 일별 평균 대기오염도 정보'를 평균냄으로써 결측치 보완 (이하 날짜)
 https://data.seoul.go.kr/dataList/OA-2218/S/1/datasetView.do
* 2019-10-08 : 23.044
* 2019-10-09 : 24.804
* 2022-02-15 : 35.063

In [123]:
# 누락된 미세먼지 농도 데이터 추가

mean_dust.append(pd.DataFrame({'일시': ['2019-10-08'], '일 미세먼지 농도(㎍/㎥)': [23.044]}), ignore_index=True)
mean_dust.append(pd.DataFrame({'일시': ['2019-10-09'], '일 미세먼지 농도(㎍/㎥)': [24.804]}), ignore_index=True)
mean_dust.append(pd.DataFrame({'일시': ['2022-02-15'], '일 미세먼지 농도(㎍/㎥)': [35.063]}), ignore_index=True)

,일시,일 미세먼지 농도(㎍/㎥)
0,2009-11-20 00:00:00,32.777778
1,2009-11-21 00:00:00,32.000000
2,2009-11-22 00:00:00,29.259259
3,2009-11-23 00:00:00,52.555556
4,2009-11-24 00:00:00,62.000000
...,...,...
5026,2023-08-28 00:00:00,15.040000
5027,2023-08-29 00:00:00,15.863636
5028,2023-08-30 00:00:00,14.000000
5029,2023-08-31 00:00:00,16.666667


In [124]:
mean_dust['일시'] = pd.to_datetime(mean_dust['일시'], format='%Y-%m-%d')
mean_dust.sort_values(by = "일시", inplace = True)
mean_dust

,일시,일 미세먼지 농도(㎍/㎥)
0,2009-11-20,32.777778
1,2009-11-21,32.000000
2,2009-11-22,29.259259
3,2009-11-23,52.555556
4,2009-11-24,62.000000
...,...,...
5025,2023-08-27,23.576923
5026,2023-08-28,15.040000
5027,2023-08-29,15.863636
5028,2023-08-30,14.000000


In [125]:
mean_dust

,일시,일 미세먼지 농도(㎍/㎥)
0,2009-11-20,32.777778
1,2009-11-21,32.000000
2,2009-11-22,29.259259
3,2009-11-23,52.555556
4,2009-11-24,62.000000
...,...,...
5025,2023-08-27,23.576923
5026,2023-08-28,15.040000
5027,2023-08-29,15.863636
5028,2023-08-30,14.000000


In [128]:
df_x['매출일자(배송완료일자)'] = pd.to_datetime(df_x['매출일자(배송완료일자)'], format='%Y-%m-%d')

In [129]:
df_x = df_x.merge(mean_dust, left_on = '매출일자(배송완료일자)', right_on = '일시', how = 'left')

### 온도

In [130]:
temperature1 = pd.read_excel('../data/외부변수/20091120-20181231_기온.xlsx')

In [131]:
temperature2 = pd.read_excel('../data/외부변수/20190101-20230831_기온.xlsx')

In [132]:
temperature = pd.concat([temperature1, temperature2])

In [133]:
mean_temperature = temperature[['일시', '평균기온(°C)']].groupby('일시').mean().reset_index()

In [134]:
mean_temperature['일시'] = pd.to_datetime(mean_temperature['일시'], format='%Y-%m-%d')
mean_temperature

,일시,평균기온(°C)
0,2009-11-20,3.035714
1,2009-11-21,2.598810
2,2009-11-22,2.825000
3,2009-11-23,5.992857
4,2009-11-24,6.148810
...,...,...
5028,2023-08-27,25.341667
5029,2023-08-28,25.096875
5030,2023-08-29,25.286458
5031,2023-08-30,22.922917


In [135]:
df_x = df_x.merge(mean_temperature, left_on = '매출일자(배송완료일자)', right_on = '일시', how = 'left')

In [136]:
df_x = df_x.drop(['일시_x', '일시_y'], axis = 1)

In [137]:
df_x = df_x.rename({'일 미세먼지 농도(㎍/㎥)' : '미세먼지', '평균기온(°C)' : '기온'}, axis = 1)

In [138]:
df_x.head(2)

,고객ID,매출일자(배송완료일자),타겟제품명,금액,멤버십고객여부,케어솔루션품목수,케어십가입여부,케어십품목수,최근3년구매금액,최근구매채널,Only&Best품목수,건강관리가전품목수,연령대,주거형태,아파트시세,아파트평형대,포인트사용빈도,보유멤버십포인트,멤버십가입경로,제휴카드보유여부,프리미엄카드보유여부,제휴가입여부,멤버십앱설치여부,거주광역명,거주시군구명,년도,월,label_2023,제품카테고리,금액_중간값,label_y,미세먼지,기온
0,34410,2022-04-29,에어컨,300만원 이상 400만원 이하,0.0,0,0,0,15KK이상20KK미만,2,1,1,30대,아파트,1~2억,20평대,미사용,0,매장,0,0,0,0.0,전북,전주시 덕진구,2022,4,0.0,에어솔루션,350,0.0,16.4,11.554737
1,34410,2020-05-25,세탁기,100만원 이상 200만원 이하,0.0,0,0,0,15KK이상20KK미만,2,1,1,30대,아파트,1~2억,20평대,미사용,0,매장,0,0,0,0.0,전북,전주시 덕진구,2020,5,0.0,청소,150,0.0,26.4,17.718947


In [139]:
def organize_purchases_per_customer_add_variable(dataframe, column_list):
    
    # 단일 고객의 최대 구매 횟수 파악
    grouped_new = dataframe.groupby('고객ID')
    max_purchases_new = grouped_new.size().max()

    label_encoding_df = {column: [] for column in column_list}

    for i in range(1, max_purchases_new + 1):
        label_encoding_df[f'년도_{i}'] = []
        label_encoding_df[f'월_{i}'] = []
        label_encoding_df[f'타겟제품명_{i}'] = []
        label_encoding_df[f'금액_{i}'] = []
        label_encoding_df[f'미세먼지_{i}'] = []
        label_encoding_df[f'기온_{i}'] = []

    for customer_id, group in tqdm(grouped_new):
        purchases = list(group[['년도', '월','타겟제품명', '금액', '미세먼지', '기온']].itertuples(index=False, name=None))
        new_row_new = create_row_from_group(group, customer_id)

        # 새 행에 판매 날짜 및 구매 추가
        for i, (year, month, product_name, amount, dust, temperature) in enumerate(purchases, 1):
            new_row_new[f'년도_{i}'] = year
            new_row_new[f'월_{i}'] = month
            new_row_new[f'타겟제품명_{i}'] = product_name
            new_row_new[f'금액_{i}'] = amount
            new_row_new[f'미세먼지_{i}'] = dust
            new_row_new[f'기온_{i}'] = temperature

        # 최대 구매수보다 적은 경우 나머지 행 채우기
        for j in range(len(purchases) + 1, max_purchases_new + 1):
            new_row_new[f'년도_{j}'] = None
            new_row_new[f'월_{j}'] = None
            new_row_new[f'타겟제품명_{j}'] = None
            new_row_new[f'금액_{j}'] = None
            new_row_new[f'미세먼지_{i}'] = None
            new_row_new[f'기온_{i}'] = None

        for key in label_encoding_df:
            label_encoding_df[key].append(new_row_new.get(key, None))

    new_structured_df_new = pd.DataFrame(label_encoding_df)
    return new_structured_df_new


In [140]:
def final_df_maker_add_variable(dataframe, col_list):
    df_new = organize_purchases_per_customer_add_variable(dataframe, col_list)
    # 가공변수 추가
    df_new = df_new.merge(df_crm[['고객ID','총구매빈도', '최대구매가격대',
       '주구매제품카테고리']], on = '고객ID', how = 'left')
    return df_new

In [141]:
col_list = [
'고객ID','멤버십고객여부', '케어솔루션품목수', '케어십가입여부', '케어십품목수', '최근3년구매금액', '최근구매채널',
'Only&Best품목수', '건강관리가전품목수', '연령대', '주거형태', '아파트시세', '아파트평형대', '포인트사용빈도',
'보유멤버십포인트', '멤버십가입경로', '제휴카드보유여부', '프리미엄카드보유여부', '제휴가입여부', '멤버십앱설치여부',
'거주광역명', '거주시군구명', 'label_y']

### Air Solution - 미세먼지, 기온

In [142]:
air2_df = df_x.loc[df_x['제품카테고리'] == '에어솔루션']

In [143]:
air2_df_new = final_df_maker_add_variable(air2_df, col_list)

100%|████████████████████████████████████| 63547/63547 [02:55<00:00, 361.58it/s]


In [144]:
air2_df_new.head(2)

,고객ID,멤버십고객여부,케어솔루션품목수,케어십가입여부,케어십품목수,최근3년구매금액,최근구매채널,Only&Best품목수,건강관리가전품목수,연령대,주거형태,아파트시세,아파트평형대,포인트사용빈도,보유멤버십포인트,멤버십가입경로,제휴카드보유여부,프리미엄카드보유여부,제휴가입여부,멤버십앱설치여부,거주광역명,거주시군구명,label_y,년도_1,월_1,타겟제품명_1,금액_1,미세먼지_1,기온_1,년도_2,월_2,타겟제품명_2,금액_2,미세먼지_2,기온_2,년도_3,월_3,타겟제품명_3,금액_3,미세먼지_3,기온_3,년도_4,월_4,타겟제품명_4,금액_4,미세먼지_4,기온_4,년도_5,월_5,타겟제품명_5,금액_5,미세먼지_5,기온_5,년도_6,월_6,타겟제품명_6,금액_6,미세먼지_6,기온_6,년도_7,월_7,타겟제품명_7,금액_7,미세먼지_7,기온_7,년도_8,월_8,타겟제품명_8,금액_8,미세먼지_8,기온_8,총구매빈도,최대구매가격대,주구매제품카테고리
0,3,Missing,0,0,0,0원,3,1,2,Missing,아파트,5~6억,30평대,미사용,0,Missing,0,0,0,Missing,경기,화성시,0.0,2020,8,공기청정기,0만원 이상 100만원 이하,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,None,None,NaN,NaN,5,250,[부엌]
1,4,Missing,3,0,0,1KK미만,4,1,2,Missing,아파트,1~2억,30평대,미사용,0,Missing,0,0,0,Missing,전북,전주시 덕진구,0.0,2018,11,공기청정기,0만원 이상 100만원 이하,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,None,None,NaN,NaN,3,150,"[부엌, 에어솔루션, 청소]"


In [145]:
air2_df_new.to_csv('../data/air_외부변수/air_df_new_값생성_미세먼지_2023제거.csv', encoding='utf-8-sig')

# 방법론 2 (사용하지 않음)
- Idea : 각 제품군 별로 방법론 1처럼 데이터프레임을 나누게 되면, 어떤 제품을 구매하고, 다른 제품군도 같이 구매하게 되는 제품 간 연관성 정보가 소실될 수 있음
- 따라서, 제품군 별로 데이터프레임을 구분하지 않고, 하나의 기존 데이터프레임으로 둠.
- 이때, Y_target은 제품군 별 구매여부(1, 0)를 매번 모델링 때마다 다르게 변화시키면서 모델 예측을 진행 ex) 동일한 데이터프레임을 가지고, 처음에는 Y_target을 부엌, 그 다음은 에어솔루션 등으로 정답지를 바꾸면서 각각 다른 모델을 도출하는 것

In [ ]:
df2 = pd.read_csv('../data/.....', index_col = 0)
df_copy = df2.copy()

## 2023년 구매 여부 컬럼

In [ ]:
# 데이터를 '년도' 기준으로 내림차순 정렬
df_copy = df_copy.sort_values(by='년도', ascending=False)

# 각 '고객ID'별로 첫 번째 행만 남기고 중복 제거
df_copy = df_copy.drop_duplicates(subset='고객ID')

In [ ]:
df_copy.loc[df_copy['년도'] == 2023, '2023년구매'] = 1
df_copy.loc[df_copy['년도'] != 2023, '2023년구매'] = 0

In [ ]:
df2 = df2.merge(df_copy[['고객ID', '2023년구매']], on='고객ID', how = 'left')

In [ ]:
df2['제품카테고리'] = df2['타겟제품명'].apply(lambda x: split_category(x))

## 2023년에 구매한 제품카테고리

In [ ]:
df_2023 = df2.loc[df2['년도'] == 2023]

In [ ]:
grouped_data = df_2023.groupby('고객ID')['제품카테고리'].apply(list).reset_index()
grouped_data

In [ ]:
df2 = df2.merge(grouped_data, on='고객ID', how = 'left')

In [ ]:
df2 = df2.rename({'제품카테고리_x' : '제품카테고리'}, axis = 1)
df2 = df2.rename({'제품카테고리_y' : '제품리스트'}, axis = 1)

In [ ]:
df2.head(1)

## 제품카테고리 별 label 컬럼 추가

In [ ]:
# 제품명과 label 컬럼명 매핑
product_label_mapping = {
    '에어솔루션': 'label_에어솔루션',
    '청소': 'label_청소',
    '부엌': 'label_부엌',
    '디스플레이': 'label_디스플레이',
    '건강관리': 'label_건강관리',
    '기타': 'label_기타'
}

# 각 행에 대해 조건에 맞게 값을 할당
for index, row in tqdm(df2.iterrows()):
    if row['2023년구매'] == 0:
        for label in product_label_mapping.values():
            df2.loc[index, label] = 0
    elif row['2023년구매'] == 1:
        purchased_products = row['제품리스트']  # 예: ['디스플레이', '청소, 빨래', ...]
        for product in purchased_products:
            label = product_label_mapping.get(product, None)
            if label:
                df2.at[index, label] = 1

In [ ]:
df2['label_에어솔루션'].fillna(0)
df2['label_청소'].fillna(0)
df2['label_부엌'].fillna(0)
df2['label_디스플레이'].fillna(0)
df2['label_기타'].fillna(0)

## 고객별 데이터 생성

In [ ]:
col_list = [
'고객ID','멤버십고객여부', '케어솔루션품목수', '케어십가입여부', '케어십품목수', '최근3년구매금액', '최근구매채널',
'Only&Best품목수', '건강관리가전품목수', '연령대', '주거형태', '아파트시세', '아파트평형대', '포인트사용빈도',
'보유멤버십포인트', '멤버십가입경로', '제휴카드보유여부', '프리미엄카드보유여부', '제휴가입여부', '멤버십앱설치여부',
'거주광역명', '거주시군구명', '제품카테고리', 'label_에어솔루션', 'label_청소', 'label_부엌',
       'label_디스플레이', 'label_건강관리', 'label_기타'] 
all_df = final_df_maker(df2, col_list)

In [ ]:
all_df

In [ ]:
# df_list = [['air', '에어솔루션'], ['clean', '청소, 빨래'], ['kitchen', '부엌'], ['display', '디스플레이'], ['health', '건강관리'], ['etc', '기타']]